In [4]:
from openai import OpenAI
import json
# 我有时候想不起来改这个key,请自觉换成您自己的
client = OpenAI(api_key="sk-ecc1b4f705dc49f8ab3bd3ce5fc9b0a3", base_url="https://api.deepseek.com")

In [2]:
def split_file(file_path, n):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    length = len(content)
    chunk_size = length // n
    chunks = [content[i:i + chunk_size] for i in range(0, length, chunk_size)]
    if len(chunks) > n:
        chunks[-2] += chunks[-1]
        chunks.pop()
    return chunks

def get_gpt_responses(chunks, sample, prompt0):
    responses = []
    i = 0
    for chunk in chunks:
        prompt = chunk + prompt0 
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": sample["system"]},
                {"role": "user", "content": sample["user"]},
                {"role": "assistant", "content": sample["assistant"]},
                {"role": "user", "content": prompt}
            ],
            stream=False,
            temperature=1.0
        )
        responses.append(response.choices[0].message.content.strip())
        i += 1
        print(f"Chunk {i} of {len(chunks)} processed.")
    return responses

def main(file_path, n, sample_path, prompt0, print_chunk = False):
    with open(sample_path, 'r', encoding='utf-8') as sample_file:
        sample = json.load(sample_file)
    
    chunks = split_file(file_path, n)
    responses = []
    i = 0
    for chunk in chunks:
        prompt = chunk + prompt0 
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": sample["system"]},
                {"role": "user", "content": sample["user"]},
                {"role": "assistant", "content": sample["assistant"]},
                {"role": "user", "content": prompt}
            ],
            stream=False,
            temperature=1.0
        )
        responses.append(response.choices[0].message.content.strip())
        i += 1
        if print_chunk:
            print(f"Chunk {i} of {len(chunks)} processed.")
    return responses

def main2(file_path, n, prompt0, print_chunk = False):
    chunks = split_file(file_path, n)
    responses = []
    i = 0
    for chunk in chunks:
        prompt = chunk + prompt0 
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "Hello! I'm an AI assistant. How can I help you today?"},
                {"role": "user", "content": "I need help with something."},
                {"role": "assistant", "content": "Sure! I'll do my best to help you."},
                {"role": "user", "content": prompt}
            ],
            stream=False,
            temperature=1.0
        )
        responses.append(response.choices[0].message.content.strip())
        i += 1
        if print_chunk:
            print(f"Chunk {i} of {len(chunks)} processed.")
    return responses

#### 生成词汇
耗时1min

In [12]:
# 调用主函数
prompt1 = "\n\n Find a new word in the text above and return its English definition in the format 'xxx : xxx' ,Please choose nouns, adjectives, adverbs, or verbs.Don't choose proper nouns"
prompt2 = "\n\n Find a word in the text above which is rare but esay to guess by the text, and return your brief guess in the format 'xxx : xxx' .\n Please choose nouns, adjectives, adverbs, or verbs.\n Don't choose proper nouns.\n Don't choose too obscure vocabulary \n brief guess consists of 3-7 words!\n Please make your guess as different as possible from the dictionary definition, so that you can guess incorrectly"
'''
a = main('a.txt', 32, 'sample1.json' , prompt1)
for i in a:
    print(i)
'''
print("---------------") 
a = main('a.txt', 40, 'sample2.json' , prompt2) 
for i in a:
    print(i)

---------------
Humbug : nonsense or deception
Residuary : remaining after death
Veneration : deep respect
Facetious : joking in a light-hearted way
Poulterers' : sellers of poultry
broadwise : wide direction
tights : close-fitting leg coverings
ponderous : heavy or weighty
abode : place of residence
preposterous : highly unreasonable
reclamation : restoring to better state
Deal : simple wooden furniture
Welsh wig : old-fashioned hairpiece
Stomach-aches : intense tuning sounds
Even-handed : fair and impartial
Pommel : strike lightly
Extinguisher-cap : cap used to smother flames
Glorious : extremely beautiful
Apoplectic : relating to excessive fatness
barnacle : marine organism that attaches to surfaces
Bustle : lively commotion
adamant : unyielding, unchangeable
Furze : thorny shrub
Loophole : narrow opening
mouldy : old and musty
Almshouse : charitable housing
Prostrate : lying flat
Excrescence : abnormal growth
Frowsy : dirty and untidy
Calico : coarse cotton cloth
Griping : tightly 

#### questions in RR
###### 这个效果不好,别用

In [ ]:
prompt3 = "If you don't fully understand the story, please ask 5 Yes/No questions with the about the story. It is best if each of questions consists of multiple logically related questions. no longer than 70 words in a.Please express it in a question"
a = main('a.txt', 1, 'sample3.json' , prompt3)

#### ideas in RR
###### 这个效果不好,别用

In [ ]:
prompt4 = "\n\n Give me 3 ideas from this reading that would be interesting to discuss,no longer than 70 words "
a = main2('a.txt', 1, prompt4)

#### 一键生成小说大意(人物介绍+切片分段大意(中英文))

In [13]:
# 设置文件名
title_of_your_md = "A Christmas Carol"
md_file_name = f"{title_of_your_md}.md"

# 准备提示
prompt5 = "\n\n  用中文帮我理一下这本小说的人物和人际关系"
prompt6 = "\n\n  用中文给这一段做一个故事梗概,多补充一点细节分点列出,并在你认为重点的地方给出英文对(请你注意格式,对照的地方一句中文对应一句英文)"

with open(md_file_name, 'w', encoding='utf-8') as md_file:
    md_file.write(f"# {title_of_your_md}\n\n")
    md_file.write("## 人物和人际关系\n\n")
    a = main2('a.txt', 1, prompt5)
    for i in a:
        md_file.write(i + "\n")
    md_file.write("<br/>\n\n")
    md_file.write("## 故事梗概\n\n")
    a = main('a.txt', 30, 'sample_for_preparing_quiz.json', prompt6, True)
    j = 0
    for i in a:
        j += 1
        md_file.write(f"### 分段{j}\n")
        md_file.write(i + "\n")
        md_file.write("<br/>\n")

print(f"输出已保存到 {md_file_name}")

Chunk 1 of 30 processed.
Chunk 2 of 30 processed.
Chunk 3 of 30 processed.
Chunk 4 of 30 processed.
Chunk 5 of 30 processed.
Chunk 6 of 30 processed.
Chunk 7 of 30 processed.
Chunk 8 of 30 processed.
Chunk 9 of 30 processed.
Chunk 10 of 30 processed.
Chunk 11 of 30 processed.
Chunk 12 of 30 processed.
Chunk 13 of 30 processed.
Chunk 14 of 30 processed.
Chunk 15 of 30 processed.
Chunk 16 of 30 processed.
Chunk 17 of 30 processed.
Chunk 18 of 30 processed.
Chunk 19 of 30 processed.
Chunk 20 of 30 processed.
Chunk 21 of 30 processed.
Chunk 22 of 30 processed.
Chunk 23 of 30 processed.
Chunk 24 of 30 processed.
Chunk 25 of 30 processed.
Chunk 26 of 30 processed.
Chunk 27 of 30 processed.
Chunk 28 of 30 processed.
Chunk 29 of 30 processed.
Chunk 30 of 30 processed.
输出已保存到 A Christmas Carol.md


#### text-sample转化器(开发用)

In [ ]:
multiline_text = """ 

"""

# 将多行文字转换为一行，换行符用\n转义
single_line_text = multiline_text.replace('\n', '\\n')

# 将结果写入到文件 b.txt
with open('b.txt', 'w', encoding='utf-8') as file:
    file.write(single_line_text)

print("文本已经写入到 b.txt 文件中。")